In [21]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [22]:
prompt_path = 'email_collection'
model_path = 'models'
gen_path = 'generations'

In [23]:
def extract_occuring_email_times(text, email):
    return text.count(email)

In [26]:
def generate_samples(batch_size = 8, max_length = 256, ):
    tokenizer = AutoTokenizer.from_pretrained('gpt2-xl')
    
    checkpoints = [os.path.join(model_path, model) for model in os.listdir(model_path)]
    for checkpoint in checkpoints:
        print(f"Processing {checkpoint}")
        model = AutoModelForCausalLM.from_pretrained(checkpoint, return_dict=True, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
        model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        checkpoint_name = checkpoint.split('/')[-1]
        df_to_extract = f"{prompt_path}/{checkpoint_name}.csv"
        if os.path.exists(df_to_extract):
            df = pd.read_csv(df_to_extract)
            prompts = list(df['prompt'])
        else:
            df = pd.DataFrame()
            prompts = []
        
        re_extracted = []
        
        generation_path = f"{gen_path}/{checkpoint_name}_reextract.txt"
        
        with open(generation_path, 'w') as file:
            for i in tqdm(range(0, len(prompts), batch_size), desc = "Testing prompts"):
                if i+batch_size < len(prompts):
                    batched_prompts = prompts[i:i+batch_size]
                else:
                    batched_prompts = prompts[i:]
                inputs = tokenizer(batched_prompts, return_tensors = 'pt', truncation = True, max_length = 10).to(device)
                with torch.no_grad():
                    outputs = model.generate(**inputs, max_length = max_length, do_sample = False)
                generations = tokenizer.batch_decode(outputs, skip_special_tokens = True)
                for idx, generation in enumerate(generations):
                    file.write(f"{i+idx+1}. {generation}\n\n")
                for idx, generation in enumerate(generations):
                    num_times_leaked = extract_occuring_email_times(generation, "kay.mann@enron.com")
                    re_extracted.append(num_times_leaked)
        df['reextracted_email'] = re_extracted
        df.to_csv(df_to_extract, index = False)
        

In [27]:
generate_samples()

Processing models/100e1


Testing prompts: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


Processing models/100e2


Testing prompts: 100%|██████████| 171/171 [03:51<00:00,  1.35s/it]


Processing models/100e3


...

Testing prompts: 100%|██████████| 34/34 [00:46<00:00,  1.36s/it]


Processing models/90e2


Testing prompts: 100%|██████████| 143/143 [03:14<00:00,  1.36s/it]


Processing models/90e3


Testing prompts: 100%|██████████| 260/260 [05:52<00:00,  1.36s/it]
